In [1]:
import pandas as pd
import os
from dateutil import parser
import traceback



### Senate Code

In [15]:
filename = 'fdsys_billstatus.xml'
import xml.etree.ElementTree as ET
tree = ET.parse(f'./congress/data/111/bills/hr/hr80/{filename}')
root = tree.getroot()
root

<Element 'billStatus' at 0x1201fb270>

In [16]:
bill = root.find('bill')

In [17]:
billNumber = bill.find('billNumber').text
createDate = parser.parse(bill.find('createDate').text)
updateDate = parser.parse(bill.find('updateDate').text)
updateDate

datetime.datetime(2021, 9, 28, 19, 57, 45, tzinfo=tzutc())

In [18]:
originChamber = bill.find('originChamber').text

In [19]:
billType = bill.find('billType').text


In [20]:
introducedDate = parser.parse(bill.find('introducedDate').text)

In [21]:
congress = bill.find('congress').text

## Committee Code




In [22]:
committeeList = []
committees = bill.find('committees').find('billCommittees')
for com in committees:
    committee = com.find('name').text
    committeeChamber = com.find('chamber').text
    committeeType = com.find('type').text
    try:
        subcommittees = com.find('subcommittees')
        subcommitteesList = []
        for sb in subcommittees:
            sbName = sb.find('name').text
            sbActivitiesList = []
            sbActivities = sb.find('activities')
            for sba in sbActivities:
                sbaName = sba.find('name').text
                sbaDate = sba.find('date').text
                sbActivitiesList.append({'name':sbaName, 'date':sbaDate})
            subcommitteesList.append({'name':sbName,'activities':sbActivitiesList})
    except:
        pass
    committeeActivities = com.find('activities')
    activities = []
    for c in committeeActivities:
        activities.append({'name':c.find('name').text,'date':c.find('date').text})
    committeeList.append({'committee':committee,'comitteeChamber':committeeChamber,'committeeType':committeeType,
                          'committeeActivities':activities, 'subcommittees':subcommitteesList})
committeeList

[{'committee': 'Environment and Public Works Committee',
  'comitteeChamber': 'Senate',
  'committeeType': 'Standing',
  'committeeActivities': [{'name': 'Reported by',
    'date': '2009-07-20T21:16:04Z'},
   {'name': 'Markup by', 'date': '2009-05-14T19:27:30Z'},
   {'name': 'Referred to', 'date': '2009-03-12T20:39:20Z'}],
  'subcommittees': []},
 {'committee': 'Natural Resources Committee',
  'comitteeChamber': 'House',
  'committeeType': 'Standing',
  'committeeActivities': [{'name': 'Referred to',
    'date': '2009-01-06T17:05:42Z'}],
  'subcommittees': [{'name': 'Fisheries, Wildlife, Oceans, and Insular Affairs Subcommittee',
    'activities': [{'name': 'Referred to', 'date': '2009-02-04T17:34:39Z'}]}]}]

## Related Bills Code

In [23]:
try:
    relatedBills = bill.find('relatedBills')
    relatedBillsList = []
    for b in relatedBills:
        latestTitle = b.find('latestTitle').text
        number = b.find('number').text
        type = b.find('type').text
        latestActionDate = parser.parse(b.find('latestAction').find('actionDate').text)
        latestActionText = b.find('latestAction').find('text').text
        relatedBillsList.append({'latestTitle':latestTitle, 'number':number,'type':type,'latestActionDate':latestActionDate,
                                 'latestActionText':latestActionText})
except:
    traceback.format_exc()
relatedBillsList

[{'latestTitle': 'Captive Primate Safety Act',
  'number': '462',
  'type': 'S',
  'latestActionDate': datetime.datetime(2009, 2, 24, 0, 0),
  'latestActionText': 'Read twice and referred to the Committee on Environment and Public Works.'}]

## Actions Code

In [24]:
actions = bill.find('actions')
actionsList = []
try:
    for a in actions:
        actionDate = a.find('actionDate').text
        actionType = a.find('type').text
        actionText = a.find('text').text
        actionsList.append({'date':actionDate, 'text':actionText, 'type':actionType})
except Exception as e:
    traceback.format_exc()
actionsList


[{'date': '2009-07-20',
  'text': 'Placed on Senate Legislative Calendar under General Orders. Calendar No. 118.',
  'type': 'Calendars'},
 {'date': '2009-07-20',
  'text': 'Committee on Environment and Public Works. Reported by Senator Boxer without amendment. With written report No. 111-53.',
  'type': 'Committee'},
 {'date': '2009-07-20',
  'text': 'Committee on Environment and Public Works. Reported by Senator Boxer without amendment. With written report No. 111-53.',
  'type': 'Committee'},
 {'date': '2009-05-14',
  'text': 'Committee on Environment and Public Works. Ordered to be reported without amendment favorably.',
  'type': 'Committee'},
 {'date': '2009-03-12',
  'text': 'Read twice and referred to the Committee on Environment and Public Works.',
  'type': 'IntroReferral'},
 {'date': '2009-02-25',
  'text': 'Received in the Senate.',
  'type': 'IntroReferral'},
 {'date': '2009-02-24',
  'text': 'Motion to reconsider laid on the table Agreed to without objection.',
  'type': 

## Sponsors Code


In [27]:
sponsors = bill.find('sponsors')
sponsorList = []
for s in sponsors:
    fullName = s.find('fullName').text
    party = s.find('party').text
    state = s.find('state').text
    sponsorList.append({'fullName':fullName, 'party':party, 'state':state})
sponsorList
cosponsorList = []
try:
    cosponsors = bill.find('cosponsors')
    for s in cosponsors:
        fullName = s.find('fullName').text
        party = s.find('party').text
        state = s.find('state').text
        cosponsorList.append({'fullName':fullName, 'party':party, 'state':state})
except:
    traceback.format_exc()
cosponsorList

[{'fullName': 'Sen. Grassley, Chuck [R-IA]', 'party': 'R', 'state': 'IA'},
 {'fullName': 'Sen. Baucus, Max [D-MT]', 'party': 'D', 'state': 'MT'},
 {'fullName': 'Sen. Snowe, Olympia J. [R-ME]', 'party': 'R', 'state': 'ME'},
 {'fullName': 'Sen. Hatch, Orrin G. [R-UT]', 'party': 'R', 'state': 'UT'}]

## Policy Area

In [28]:
policyArea = bill.find('policyArea').find('name').text
policyArea

'Health'

## Summaries Code


In [29]:
summary = bill.find('summaries').find('billSummaries')[0].find('text').text
summary

'Prescription Drug and Medicare Improvement Act of 2003 - Expresses the sense of the Congress that the Congress should enact, and the President should sign, legislation to amend title XVIII (Medicare) of the Social Security Act to make improvements in the Medicare program and to provide prescription drug coverage under the Medicare program.'

## Title Code

In [30]:
title = bill.find('title').text
title

'Prescription Drug and Medicare Improvement Act of 2003'

In [31]:

import sqlalchemy as db
db = db.create_engine('postgresql://sanjee@localhost/data', echo=True)

In [32]:
from sqlalchemy import Text, Date, Table, Column, Integer, String, MetaData
from sqlalchemy.dialects.postgresql import JSONB
meta = MetaData()

In [33]:
s = Table(
   's', meta,
   Column('billNumber', Integer),
   Column('originChamber', Text),
   Column('billType', Text),
   Column('introducedDate', Date),
   Column('congress', Integer),
   Column('committees', JSONB),
   Column('relatedBills', JSONB),
   Column('actions', JSONB),
   Column('sponsors', JSONB),
   Column('cosponsors', JSONB),
   Column('policyArea', Text),
   Column('summary', Text),
   Column('title', Text),
)
meta.create_all(db)

2021-11-23 03:24:27,823 INFO sqlalchemy.engine.Engine select version()
2021-11-23 03:24:27,824 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-23 03:24:27,826 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-23 03:24:27,827 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-23 03:24:27,829 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-11-23 03:24:27,830 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-23 03:24:27,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-23 03:24:27,833 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-11-23 03:24:27,834 INFO sqlalchemy.engine.Engine [generated in 0.00089s] {'name': 's'}
2021-11-23 03:24:27,841 INFO sqlalchemy.engine.Engine COMMIT


In [34]:
ins = s.insert().values(billNumber=billNumber,originChamber=originChamber, billType=billType, introducedDate=introducedDate,
                        congress=congress, committees=committeeList, actions=actionsList, sponsors=sponsorList, cosponsors=cosponsorList,
                        policyArea=policyArea, summary=summary, title=title)

In [35]:
conn = db.connect()

In [36]:
result = conn.execute(ins)

2021-11-23 03:24:31,626 INFO sqlalchemy.engine.Engine INSERT INTO s ("billNumber", "originChamber", "billType", "introducedDate", congress, committees, actions, sponsors, cosponsors, "policyArea", summary, title) VALUES (%(billNumber)s, %(originChamber)s, %(billType)s, %(introducedDate)s, %(congress)s, %(committees)s, %(actions)s, %(sponsors)s, %(cosponsors)s, %(policyArea)s, %(summary)s, %(title)s)
2021-11-23 03:24:31,627 INFO sqlalchemy.engine.Engine [generated in 0.00153s] {'billNumber': '1', 'originChamber': 'Senate', 'billType': 'S', 'introducedDate': datetime.datetime(2003, 6, 11, 0, 0), 'congress': '108', 'committees': '[{"committee": "Finance Committee", "comitteeChamber": "Senate", "committeeType": "Standing", "committeeActivities": [{"name": "Reported by", "date": ... (11 characters truncated) ... 3T16:38:34Z"}, {"name": "Markup by", "date": "2003-06-12T16:15:53Z"}, {"name": "Referred to", "date": "2003-06-11T23:57:41Z"}], "subcommittees": []}]', 'actions': '[{"date": "2003-0

## JSON Parsing Code for archived bills

{'actions': [{'acted_at': '1973-01-03',
   'committee': 'House Committee on Ways and Means',
   'references': [],
   'status': 'REFERRED',
   'text': 'Referred to House Committee on Ways and Means.',
   'type': 'referral'}],
 'amendments': [],
 'bill_id': 'hr1-93',
 'bill_type': 'hr',
 'committees': [{'activity': ['referral', 'in committee'],
   'committee': 'House Ways and Means',
   'committee_id': 'HSWM'}],
 'congress': '93',
 'cosponsors': [],
 'enacted_as': None,
 'history': {'awaiting_signature': False, 'enacted': False, 'vetoed': False},
 'introduced_at': '1973-01-03',
 'number': '1',
 'official_title': 'A bill to establish a new program of health care delivery and comprehensive health care delivery and comprehensive health care benefits (including catastrophic coverage), to be available to aged persons, and to employed, unemployed, and low-income individuals, at a cost related to their income.',
 'popular_title': None,
 'related_bills': [],
 'short_title': 'National Health Care

In [13]:
import json
from models import conn, tables
# for t in tables:
#     congressNumbers = os.listdir('./data')
#     for cN in congressNumbers:
#         bills = os.listdir(f'./data/{cN}/bills/{t}')
#         for b in bills:
#             filename = f'./data/{cN}/bills/{t}/{b}/data.json'
data = json.load(open('./data/93/bills/s/s16/data.json', 'r'))
data

{'actions': [{'acted_at': '1973-01-04',
   'committee': 'Senate Committee on Commerce',
   'references': [],
   'status': 'REFERRED',
   'text': 'Referred to Senate Committee on Commerce.',
   'type': 'referral'}],
 'amendments': [],
 'bill_id': 's16-93',
 'bill_type': 's',
 'committees': [{'activity': ['referral', 'in committee'],
   'committee': 'Senate Commerce',
   'committee_id': 'SSCM'}],
 'congress': '93',
 'cosponsors': [{'district': None,
   'name': 'Nunn, Sam',
   'sponsored_at': '1973-01-03',
   'state': 'GA',
   'thomas_id': '01413',
   'title': 'Sen',
   'withdrawn_at': None}],
 'enacted_as': None,
 'history': {'awaiting_signature': False, 'enacted': False, 'vetoed': False},
 'introduced_at': '1973-01-04',
 'number': '16',
 'official_title': 'A bill to amend the Communications Act of 1934 to establish orderly procedures for the consideration of application for renewal of broadcast licenses.',
 'popular_title': None,
 'related_bills': [],
 'short_title': 'Broadcast License 

In [10]:
billNumber = data['number']
billNumber
billType = str(data['bill_type']).toUpperCase()
introducedDate = data['introduced_at']
congress = data['congress']

In [ ]:
committees = data['committees']
committeeList = []
try:
    for com in committees:
        committee = data['committee']
        committeeList.append(
            {'committee': committee})
except:
    pass
title = data['short_title']
summary = data['summary']['text']
actions = data['actions']
actionList = []
for a in actions:
    actionList.append({'date': a['acted_at'], 'text': a['text'], 'type': a['type']})

## Sponsors Code


In [ ]:
sponsorList = []
sponsor = data['sponsor']
if sponsor['title'] == 'Sen':
    sponsorTitle = f"{sponsor['title']} {sponsor['name']} [{sponsor['state']}]"
else:
    sponsorTitle = f"{sponsor['title']} {sponsor['name']} [{sponsor['state']}-{sponsor['district']}]"
sponsorList.append({'fullName': sponsorTitle})
sponsorList
cosponsorList = []
try:
    cosponsors = data['cosponsors']
    for sponsor in cosponsors:
        if sponsor['title'] == 'Sen':
            sponsorTitle = f"{sponsor['title']} {sponsor['name']} [{sponsor['state']}]"
        else:
            sponsorTitle = f"{sponsor['title']} {sponsor['name']} [{sponsor['state']}-{sponsor['district']}]"
       cosponsorList.append({'fullName': sponsorTitle})
except:
    traceback.format_exc()

ins = s.insert().values(billNumber=billNumber,billType=billType, introducedDate=introducedDate,
                        congress=congress, committees=committeeList, actions=actionsList, sponsors=sponsorList, cosponsors=cosponsorList,
                        title=title)